In [1]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'mmlu_math.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,question,subject,choices,answer
0,consensus,Solve the following math problem and select th...,Solve the following math problem and select th...,[d],[d],multiple_choice,"If a pentagon P with vertices at (– 2, – 4), (...",high_school_mathematics,"[(0, – 3), (4, 1), (2, 2), (– 4, –2)]",3
1,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[c],[c],multiple_choice,The length of a rectangle is twice its width. ...,high_school_mathematics,"[2500, 2, 50, 25]",2
2,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[a],[a],multiple_choice,"A positive integer n is called “powerful” if, ...",high_school_mathematics,"[392, 336, 300, 297]",0
3,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[b],[b],multiple_choice,"At breakfast, lunch, and dinner, Joe randomly ...",high_school_mathematics,"[\frac{7}{9}, \frac{8}{9}, \frac{5}{9}, \frac{...",1
4,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[c],[c],multiple_choice,Suppose $f(x)$ is a function that has this pro...,high_school_mathematics,"[(-inf, 10), (-inf, 9), (-inf, 8), (-inf, 7)]",2


In [3]:
df['platinum_prompt'].value_counts()

platinum_prompt
Solve the following math problem and select the correct multiple-choice answer:\n    \nWhen the expression $-2x^2-20x-53$ is written in the form $a(x+d)^2+e$, where $a$, $d$, and $e$ are constants, what is the sum $a+d+e$?\n\nOptions:\nA) -80\nB) 0\nC) 6\nD) 53\n\nThink step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                                                                                                                      1
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nThink step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.         

In [4]:
import re

def double_string(match):
    return re.sub(r'\n\n.*$', '\n\n', match.group(1)) + "Read the question again: " + match.group(1).lstrip()

df['RE2'] = df['platinum_prompt'].apply(lambda x: re.sub(r'(\n    \n.*?Think step-by-step)', double_string, x, flags=re.DOTALL))
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].apply(lambda x: re.sub(r'(\n    \n.*?Then, provide the final answer in the format)', double_string, x, flags=re.DOTALL))

In [27]:
df['RE2_no_cot'].value_counts()

RE2_no_cot
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nRead the question again: If a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nThen, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                                                                              1
Solve the following math problem and select the correct multiple-choice answer:\n    \nSix 6-sided dice are rolled. What is the probability that three of the dice show prime numbers and

In [49]:
df['platinum_prompt_no_cot'].value_counts()

platinum_prompt_no_cot
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nThen, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                         1
Solve the following math problem and select the correct multiple-choice answer:\n    \nSix 6-sided dice are rolled. What is the probability that three of the dice show prime numbers and the rest show composite numbers?\n\nOptions:\nA) \frac{125}{2916}\nB) \frac{5}{16}\nC) \frac{5}{54}\nD) \frac{5}{55}\n\nThen, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                                                 1
Solve the f

In [5]:
df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(?=Then, provide the final answer in the format)',
    r'\1Read the question again first. ',
    regex=True
)
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(?=Think step-by-step.)',
    r'\1Read the question again first. ',
    regex=True
)

In [6]:
df['sRE2'].value_counts()

sRE2
Solve the following math problem and select the correct multiple-choice answer:\n    \nWhen the expression $-2x^2-20x-53$ is written in the form $a(x+d)^2+e$, where $a$, $d$, and $e$ are constants, what is the sum $a+d+e$?\n\nOptions:\nA) -80\nB) 0\nC) 6\nD) 53\n\nRead the question again first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                                                                                                                      1
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nRead the question again first. Think step-by-step. Then, provide the final answer in the format "Answer

In [ ]:
df['sRE2_m1_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(?=Then, provide the final answer in the format)',
    r'\1Read the previous content again first. ',
    regex=True
)
df['sRE2_m1'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(?=Think step-by-step.)',
    r'\1Read the previous content again first. ',
    regex=True
)

In [8]:
df['sRE2_m1'].value_counts()

sRE2_m1
Solve the following math problem and select the correct multiple-choice answer:\n    \nWhen the expression $-2x^2-20x-53$ is written in the form $a(x+d)^2+e$, where $a$, $d$, and $e$ are constants, what is the sum $a+d+e$?\n\nOptions:\nA) -80\nB) 0\nC) 6\nD) 53\n\nRead the previous content again first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                                                                                                                      1
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nRead the previous content again first. Think step-by-step. Then, provide the final answer in

In [9]:
df['sRE2_m2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(?=Then, provide the final answer in the format)',
    r'\1Read the math problem again first. ',
    regex=True
)
df['sRE2_m2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(?=Think step-by-step.)',
    r'\1Read the math problem again first. ',
    regex=True
)


In [10]:
df['sRE2_m2'].value_counts()

sRE2_m2
Solve the following math problem and select the correct multiple-choice answer:\n    \nWhen the expression $-2x^2-20x-53$ is written in the form $a(x+d)^2+e$, where $a$, $d$, and $e$ are constants, what is the sum $a+d+e$?\n\nOptions:\nA) -80\nB) 0\nC) 6\nD) 53\n\nRead the math problem again first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                                                                                                                      1
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nRead the math problem again first. Think step-by-step. Then, provide the final answer in the for

In [ ]:
# df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(\n\s*\n)(?=Then, provide the final answer in the format)',
#     r'\1Summarize the question first. ',
#     regex=True
# )
# df['sum'] = df['platinum_prompt'].str.replace(
#     r'(\n\s*\n)(?=Think step-by-step.)',
#     r'\1Summarize the question first. ',
#     regex=True
# )

In [ ]:
# df['sum_no_cot'].value_counts()

sum_no_cot
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nSummarize the question first. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                         1
Solve the following math problem and select the correct multiple-choice answer:\n    \nSix 6-sided dice are rolled. What is the probability that three of the dice show prime numbers and the rest show composite numbers?\n\nOptions:\nA) \frac{125}{2916}\nB) \frac{5}{16}\nC) \frac{5}{54}\nD) \frac{5}{55}\n\nSummarize the question first. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                              

In [ ]:
# df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(\n\s*\n)(?=Then, provide the final answer in the format)',
#     r'\1Organize the question into a table first. ',
#     regex=True
# )
# df['table'] = df['platinum_prompt'].str.replace(
#     r'(\n\s*\n)(?=Think step-by-step.)',
#     r'\1Organize the question into a table first. ',
#     regex=True
# )

In [ ]:
# df['table'].value_counts()

table
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nOrganize the question into a table first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                         1
Solve the following math problem and select the correct multiple-choice answer:\n    \nSix 6-sided dice are rolled. What is the probability that three of the dice show prime numbers and the rest show composite numbers?\n\nOptions:\nA) \frac{125}{2916}\nB) \frac{5}{16}\nC) \frac{5}{54}\nD) \frac{5}{55}\n\nOrganize the question into a table first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct l

In [ ]:
# df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(\n\s*\n)(?=Then, provide the final answer in the format)',
#     r'\1Organize the question into a graph structure first. ',
#     regex=True
# )
# df['graph'] = df['platinum_prompt'].str.replace(
#     r'(\n\s*\n)(?=Think step-by-step.)',
#     r'\1Organize the question into a graph structure first. ',
#     regex=True
# )

In [ ]:
# df['graph'].value_counts()

graph
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nOrganize the question into a graph structure first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                         1
Solve the following math problem and select the correct multiple-choice answer:\n    \nSix 6-sided dice are rolled. What is the probability that three of the dice show prime numbers and the rest show composite numbers?\n\nOptions:\nA) \frac{125}{2916}\nB) \frac{5}{16}\nC) \frac{5}{54}\nD) \frac{5}{55}\n\nOrganize the question into a graph structure first. Think step-by-step. Then, provide the final answer in the format "Answer: X" wher

In [ ]:
# df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(\n\s*\n)(?=Then, provide the final answer in the format)',
#     r'\1Summarize the question in bullet points first. ',
#     regex=True
# )
# df['bullet_point'] = df['platinum_prompt'].str.replace(
#     r'(\n\s*\n)(?=Think step-by-step.)',
#     r'\1Summarize the question in bullet points first. ',
#     regex=True
# )

In [ ]:
# df['bullet_point'].value_counts()

bullet_point
Solve the following math problem and select the correct multiple-choice answer:\n    \nIf a pentagon P with vertices at (– 2, – 4), (– 4, 1), (–1, 4), (2, 4), and (3, 0) is reflected across the line y = x to get a new pentagon, P’, then one of the vertices of P’ is\n\nOptions:\nA) (0, – 3)\nB) (4, 1)\nC) (2, 2)\nD) (– 4, –2)\n\nSummarize the question in bullet points first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.                                                         1
Solve the following math problem and select the correct multiple-choice answer:\n    \nSix 6-sided dice are rolled. What is the probability that three of the dice show prime numbers and the rest show composite numbers?\n\nOptions:\nA) \frac{125}{2916}\nB) \frac{5}{16}\nC) \frac{5}{54}\nD) \frac{5}{55}\n\nSummarize the question in bullet points first. Think step-by-step. Then, provide the final answer in the format "Answer: X" where X

In [11]:
df.to_parquet('mmlu_m.parquet')

In [12]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,question,subject,choices,answer,RE2,RE2_no_cot,sRE2_no_cot,sRE2,sRE2_m1_no_cot,sRE2_m1,sRE2_m2_no_cot,sRE2_m2
0,consensus,Solve the following math problem and select th...,Solve the following math problem and select th...,[d],[d],multiple_choice,"If a pentagon P with vertices at (– 2, – 4), (...",high_school_mathematics,"[(0, – 3), (4, 1), (2, 2), (– 4, –2)]",3,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...
1,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[c],[c],multiple_choice,The length of a rectangle is twice its width. ...,high_school_mathematics,"[2500, 2, 50, 25]",2,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...
2,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[a],[a],multiple_choice,"A positive integer n is called “powerful” if, ...",high_school_mathematics,"[392, 336, 300, 297]",0,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...
3,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[b],[b],multiple_choice,"At breakfast, lunch, and dinner, Joe randomly ...",high_school_mathematics,"[\frac{7}{9}, \frac{8}{9}, \frac{5}{9}, \frac{...",1,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...
4,verified,Solve the following math problem and select th...,Solve the following math problem and select th...,[c],[c],multiple_choice,Suppose $f(x)$ is a function that has this pro...,high_school_mathematics,"[(-inf, 10), (-inf, 9), (-inf, 8), (-inf, 7)]",2,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...,Solve the following math problem and select th...
